In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

english_file = 'english-corpus.txt'
urdu_file = 'urdu-corpus.txt'

with open(english_file, 'r', encoding='utf-8') as file:
    english_sentences = file.readlines()

with open(urdu_file, 'r', encoding='utf-8') as file:
    urdu_sentences = file.readlines()

assert len(english_sentences) == len(urdu_sentences), "Mismatch in number of sentences between English and Urdu files."
data = pd.DataFrame({'English': [sentence.strip() for sentence in english_sentences],
                     'Urdu': [sentence.strip() for sentence in urdu_sentences]})

english_tokenizer = Tokenizer()
urdu_tokenizer = Tokenizer()

english_tokenizer.fit_on_texts(data['English'])
urdu_tokenizer.fit_on_texts(data['Urdu'])

english_sequences = english_tokenizer.texts_to_sequences(data['English'])
urdu_sequences = urdu_tokenizer.texts_to_sequences(data['Urdu'])

max_length = 20
english_padded = pad_sequences(english_sequences, maxlen=max_length, padding='post', truncating='post')
urdu_padded = pad_sequences(urdu_sequences, maxlen=max_length, padding='post', truncating='post')

decoder_input_data = urdu_padded[:, :-1]
decoder_output_data = urdu_padded[:, 1:]

decoder_input_data = np.pad(decoder_input_data, ((0, 0), (0, 1)), mode='constant', constant_values=0)
decoder_output_data = np.pad(decoder_output_data, ((0, 0), (0, 1)), mode='constant', constant_values=0)

embedding_dim = 100
rnn_units = 256

vocab_size_eng = len(english_tokenizer.word_index) + 1
vocab_size_urdu = len(urdu_tokenizer.word_index) + 1

encoder_inputs = Input(shape=(max_length,))
encoder_embedding = Embedding(vocab_size_eng, embedding_dim, input_length=max_length)(encoder_inputs)
encoder_rnn = SimpleRNN(rnn_units, return_state=True)(encoder_embedding)
encoder_outputs, state_h = encoder_rnn

decoder_inputs = Input(shape=(max_length,))
decoder_embedding = Embedding(vocab_size_urdu, embedding_dim, input_length=max_length)(decoder_inputs)
decoder_rnn = SimpleRNN(rnn_units, return_sequences=True)(decoder_embedding, initial_state=state_h)
decoder_outputs = TimeDistributed(Dense(vocab_size_urdu, activation='softmax'))(decoder_rnn)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit([english_padded, decoder_input_data], decoder_output_data,
                    epochs=50,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[early_stopping])

print("Training Complete")


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_18        │ (None, 20, 100)   │    567,900 │ input_layer_18[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_19        │ (None, 20, 100)   │    593,000 │ input_layer_19[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_18       │ [(None, 256),     │     91,392 │ embedding_18[0][… │
│ (SimpleRNN)         │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_19       │ (None, 20, 256)   │     91,392 │ embedding_19[0][… │
│ (SimpleRNN)         │                   │            │ simple_rnn_18[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_9  │ (None, 20, 5930)  │  1,524,010 │ simple_rnn_19[0]… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,867,694 (10.94 MB)

 Trainable params: 2,867,694 (10.94 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 69s 216ms/step - accuracy: 0.7792 - loss: 2.3474 - val_accuracy: 0.8138 - val_loss: 1.2028
Epoch 2/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 77s 249ms/step - accuracy: 0.8165 - loss: 1.1521 - val_accuracy: 0.8240 - val_loss: 1.0940
Epoch 3/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 73s 237ms/step - accuracy: 0.8286 - loss: 1.0323 - val_accuracy: 0.8332 - val_loss: 1.0154
Epoch 4/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 71s 232ms/step - accuracy: 0.8389 - loss: 0.9359 - val_accuracy: 0.8400 - val_loss: 0.9707
Epoch 5/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 70s 228ms/step - accuracy: 0.8457 - loss: 0.8621 - val_accuracy: 0.8427 - val_loss: 0.9453
Epoch 6/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 70s 229ms/step - accuracy: 0.8496 - loss: 0.8104 - val_accuracy: 0.8466 - val_loss: 0.9296
Epoch 7/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 73s 238ms/step - accuracy: 0.8546 - loss: 0.7651 - val_accuracy: 0.8469 - val_loss: 0.9235
Epoch 8/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 70s 226ms/step - accuracy: 0.8585 - loss: 0

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, Dense, Bidirectional, RepeatVector, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

with open('english-corpus.txt', 'r', encoding='utf-8') as f:
    english_sentences = [line.strip() for line in f.readlines()]

with open('urdu-corpus.txt', 'r', encoding='utf-8') as f:
    urdu_sentences = [line.strip() for line in f.readlines()]

assert len(english_sentences) == len(urdu_sentences), "Sentence count mismatch."

eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)

urdu_tokenizer = Tokenizer()
urdu_tokenizer.fit_on_texts(urdu_sentences)
urdu_sequences = urdu_tokenizer.texts_to_sequences(urdu_sentences)

max_len = 20
encoder_input_data = pad_sequences(eng_sequences, maxlen=max_len, padding='post')
decoder_output_data = pad_sequences(urdu_sequences, maxlen=max_len, padding='post')

decoder_input_data = np.zeros_like(decoder_output_data)
decoder_input_data[:, 1:] = decoder_output_data[:, :-1]

eng_vocab_size = len(eng_tokenizer.word_index) + 1
urdu_vocab_size = len(urdu_tokenizer.word_index) + 1

embedding_dim = 100
rnn_units = 256

encoder_inputs = Input(shape=(max_len,))
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=embedding_dim, input_length=max_len)(encoder_inputs)
encoder_outputs = Bidirectional(SimpleRNN(rnn_units, return_sequences=False))(encoder_embedding)

context_vector = RepeatVector(max_len)(encoder_outputs)
decoder_inputs = Input(shape=(max_len,))
decoder_embedding = Embedding(input_dim=urdu_vocab_size, output_dim=embedding_dim, input_length=max_len)(decoder_inputs)

decoder_combined = decoder_embedding
decoder_rnn = SimpleRNN(rnn_units * 2, return_sequences=True)(decoder_combined)
decoder_outputs = TimeDistributed(Dense(urdu_vocab_size, activation='softmax'))(decoder_rnn)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_output_data[..., np.newaxis],
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping]
)

print("Training Complete.")

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_29      │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_29        │ (None, 20, 100)   │    593,000 │ input_layer_29[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_25       │ (None, 20, 512)   │    313,856 │ embedding_29[0][… │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_28      │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_13 │ (None, 20, 5930)  │  3,042,090 │ simple_rnn_25[0]… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,948,946 (15.06 MB)

 Trainable params: 3,948,946 (15.06 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 102s 324ms/step - accuracy: 0.7435 - loss: 2.2680 - val_accuracy: 0.7735 - val_loss: 1.3972
Epoch 2/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 100s 327ms/step - accuracy: 0.7775 - loss: 1.3362 - val_accuracy: 0.7854 - val_loss: 1.2676
Epoch 3/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 98s 321ms/step - accuracy: 0.7898 - loss: 1.1929 - val_accuracy: 0.7945 - val_loss: 1.2010
Epoch 4/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 108s 353ms/step - accuracy: 0.7991 - loss: 1.0940 - val_accuracy: 0.7988 - val_loss: 1.1695
Epoch 5/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 102s 334ms/step - accuracy: 0.8048 - loss: 1.0276 - val_accuracy: 0.8022 - val_loss: 1.1497
Epoch 6/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 103s 337ms/step - accuracy: 0.8098 - loss: 0.9716 - val_accuracy: 0.8037 - val_loss: 1.1411
Epoch 7/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 111s 360ms/step - accuracy: 0.8150 - loss: 0.9255 - val_accuracy: 0.8044 - val_loss: 1.1410
Epoch 8/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 115s 375ms/step - accuracy: 0.8190 - 

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

english_file = 'english-corpus.txt'
urdu_file = 'urdu-corpus.txt'

with open(english_file, 'r', encoding='utf-8') as file:
    english_sentences = file.readlines()

with open(urdu_file, 'r', encoding='utf-8') as file:
    urdu_sentences = file.readlines()

assert len(english_sentences) == len(urdu_sentences), "Mismatch in number of sentences between English and Urdu files."
data = pd.DataFrame({'English': [sentence.strip() for sentence in english_sentences],
                     'Urdu': [sentence.strip() for sentence in urdu_sentences]})

english_tokenizer = Tokenizer()
urdu_tokenizer = Tokenizer()

english_tokenizer.fit_on_texts(data['English'])
urdu_tokenizer.fit_on_texts(data['Urdu'])

english_sequences = english_tokenizer.texts_to_sequences(data['English'])
urdu_sequences = urdu_tokenizer.texts_to_sequences(data['Urdu'])

max_length = 20
english_padded = pad_sequences(english_sequences, maxlen=max_length, padding='post', truncating='post')
urdu_padded = pad_sequences(urdu_sequences, maxlen=max_length, padding='post', truncating='post')

decoder_input_data = urdu_padded[:, :-1]
decoder_output_data = urdu_padded[:, 1:]

decoder_input_data = np.pad(decoder_input_data, ((0, 0), (0, 1)), mode='constant', constant_values=0)
decoder_output_data = np.pad(decoder_output_data, ((0, 0), (0, 1)), mode='constant', constant_values=0)

embedding_dim = 100
lstm_units = 256

vocab_size_eng = len(english_tokenizer.word_index) + 1
vocab_size_urdu = len(urdu_tokenizer.word_index) + 1

encoder_inputs = Input(shape=(max_length,))
encoder_embedding = Embedding(vocab_size_eng, embedding_dim, input_length=max_length)(encoder_inputs)
encoder_lstm, forward_h, forward_c = LSTM(lstm_units, return_state=True)(encoder_embedding)
encoder_state = [forward_h, forward_c]

decoder_inputs = Input(shape=(max_length,))
decoder_embedding = Embedding(vocab_size_urdu, embedding_dim, input_length=max_length)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True)(decoder_embedding, initial_state=encoder_state)
decoder_outputs = TimeDistributed(Dense(vocab_size_urdu, activation='softmax'))(decoder_lstm)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit([english_padded, decoder_input_data], decoder_output_data,
                    epochs=50,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[early_stopping])

print("Training Complete")

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_23      │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_22        │ (None, 20, 100)   │    567,900 │ input_layer_22[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_23        │ (None, 20, 100)   │    593,000 │ input_layer_23[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    365,568 │ embedding_22[0][… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 20, 256)   │    365,568 │ embedding_23[0][… │
│                     │                   │            │ lstm[0][1],       │
│                     │                   │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_11 │ (None, 20, 5930)  │  1,524,010 │ lstm_1[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,416,046 (13.03 MB)

 Trainable params: 3,416,046 (13.03 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 82s 257ms/step - accuracy: 0.7853 - loss: 2.4296 - val_accuracy: 0.8106 - val_loss: 1.2333
Epoch 2/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 92s 300ms/step - accuracy: 0.8118 - loss: 1.1986 - val_accuracy: 0.8177 - val_loss: 1.1567
Epoch 3/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 129s 256ms/step - accuracy: 0.8206 - loss: 1.1138 - val_accuracy: 0.8242 - val_loss: 1.0924
Epoch 4/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 85s 276ms/step - accuracy: 0.8281 - loss: 1.0373 - val_accuracy: 0.8305 - val_loss: 1.0403
Epoch 5/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 81s 262ms/step - accuracy: 0.8353 - loss: 0.9689 - val_accuracy: 0.8375 - val_loss: 0.9933
Epoch 6/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 79s 257ms/step - accuracy: 0.8428 - loss: 0.9014 - val_accuracy: 0.8444 - val_loss: 0.9460
Epoch 7/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 75s 244ms/step - accuracy: 0.8507 - loss: 0.8433 - val_accuracy: 0.8502 - val_loss: 0.9010
Epoch 8/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 84s 273ms/step - accuracy: 0.8589 - loss: 

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, Dropout, MultiHeadAttention, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

english_file = 'english-corpus.txt'
urdu_file = 'urdu-corpus.txt'

with open(english_file, 'r', encoding='utf-8') as file:
    english_sentences = file.readlines()

with open(urdu_file, 'r', encoding='utf-8') as file:
    urdu_sentences = file.readlines()

assert len(english_sentences) == len(urdu_sentences), "Mismatch in number of sentences between English and Urdu files."
data = pd.DataFrame({'English': [sentence.strip() for sentence in english_sentences],
                     'Urdu': [sentence.strip() for sentence in urdu_sentences]})

english_tokenizer = Tokenizer()
urdu_tokenizer = Tokenizer()

english_tokenizer.fit_on_texts(data['English'])
urdu_tokenizer.fit_on_texts(data['Urdu'])

english_sequences = english_tokenizer.texts_to_sequences(data['English'])
urdu_sequences = urdu_tokenizer.texts_to_sequences(data['Urdu'])

max_length = 20
english_padded = pad_sequences(english_sequences, maxlen=max_length, padding='post', truncating='post')
urdu_padded = pad_sequences(urdu_sequences, maxlen=max_length, padding='post', truncating='post')

decoder_input_data = urdu_padded[:, :-1]
decoder_output_data = urdu_padded[:, 1:]
decoder_input_data = np.pad(decoder_input_data, ((0, 0), (0, 1)), mode='constant', constant_values=0)
decoder_output_data = np.pad(decoder_output_data, ((0, 0), (0, 1)), mode='constant', constant_values=0)

embedding_dim = 100
num_heads = 8
ff_dim = 512
num_layers = 4

vocab_size_eng = len(english_tokenizer.word_index) + 1
vocab_size_urdu = len(urdu_tokenizer.word_index) + 1

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention = Add()([inputs, attention])
    attention = LayerNormalization()(attention)
    
    ffn = Dense(ff_dim, activation='relu')(attention)
    ffn = Dense(inputs.shape[-1])(ffn)
    ffn = Dropout(dropout)(ffn)
    ffn = Add()([attention, ffn])
    ffn = LayerNormalization()(ffn)
    
    return ffn

encoder_inputs = Input(shape=(max_length,))
encoder_embedding = Embedding(vocab_size_eng, embedding_dim, input_length=max_length)(encoder_inputs)

x = encoder_embedding
for _ in range(num_layers):
    x = transformer_encoder(x, head_size=embedding_dim, num_heads=num_heads, ff_dim=ff_dim)

decoder_inputs = Input(shape=(max_length,))
decoder_embedding = Embedding(vocab_size_urdu, embedding_dim, input_length=max_length)(decoder_inputs)

y = decoder_embedding
for _ in range(num_layers):
    y = transformer_encoder(y, head_size=embedding_dim, num_heads=num_heads, ff_dim=ff_dim)
decoder_outputs = Dense(vocab_size_urdu, activation='softmax')(y)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit([english_padded, decoder_input_data], decoder_output_data,
                    epochs=50,
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[early_stopping])

print("Training Complete")

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25      │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_25        │ (None, 20, 100)   │    593,000 │ input_layer_25[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 20, 100)   │    322,500 │ embedding_25[0][… │
│ (MultiHeadAttentio… │                   │            │ embedding_25[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 20, 100)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 20, 100)   │          0 │ embedding_25[0][… │
│                     │                   │            │ dropout_13[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 20, 100)   │        200 │ add_8[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 20, 512)   │     51,712 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 20, 100)   │     51,300 │ dense_20[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 20, 100)   │          0 │ dense_21[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 20, 100)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 20, 100)   │        200 │ add_9[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 20, 100)   │    322,500 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 20, 100)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_10 (Add)        │ (None, 20, 100)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_16[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 20, 100)   │        200 │ add_10[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 20, 512)   │     51,712 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 20, 100)   │     51,300 │ dense_22[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 20, 100)   │          0 │ dense_23[0][0]    │
│ (Dropout)           │                   │            │                 

 Total params: 2,895,578 (11.05 MB)

 Trainable params: 2,895,578 (11.05 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 111s 322ms/step - accuracy: 0.7887 - loss: 2.9755 - val_accuracy: 0.8734 - val_loss: 0.8676
Epoch 2/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 104s 338ms/step - accuracy: 0.8889 - loss: 0.7501 - val_accuracy: 0.9209 - val_loss: 0.5571
Epoch 3/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 94s 306ms/step - accuracy: 0.9335 - loss: 0.4431 - val_accuracy: 0.9438 - val_loss: 0.4037
Epoch 4/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 101s 330ms/step - accuracy: 0.9567 - loss: 0.2786 - val_accuracy: 0.9556 - val_loss: 0.3287
Epoch 5/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 106s 344ms/step - accuracy: 0.9695 - loss: 0.1871 - val_accuracy: 0.9595 - val_loss: 0.2998
Epoch 6/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 97s 315ms/step - accuracy: 0.9785 - loss: 0.1232 - val_accuracy: 0.9621 - val_loss: 0.2825
Epoch 7/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 105s 340ms/step - accuracy: 0.9841 - loss: 0.0874 - val_accuracy: 0.9632 - val_loss: 0.2809
Epoch 8/50
307/307 ━━━━━━━━━━━━━━━━━━━━ 103s 334ms/step - accuracy: 0.9878 - l